In [1]:
import serial, threading, sys, time
sys.path.append('/home/pi/Desktop/ADR_NX/Utils')
#sys.path.append('C:/GoogleDrive/SPT/SW/ADR_NX/Utils')
from HCLib import HeaterController
import matplotlib.pyplot as plt
from adr_PID import ADR_PID
import numpy as np

In [2]:
debug = 1
run_tests = 1
HC = HeaterController.HeaterController()
HC.establish_comms()
HC.debug = 0

HCDBG: Port ready: Serial<id=0x6fd9fbd0, open=False>(port=None, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=0.2, xonxoff=False, rtscts=False, dsrdtr=False)
HCDBG: Platform (linux2) -> device list is ['/dev/ADR_heatercontrol', '/dev/ttyUSB2']
HCDBG: Starting device search
HCDBG: Port /dev/ADR_heatercontrol did not work, exception: [Errno 2] could not open port /dev/ADR_heatercontrol: [Errno 2] No such file or directory: '/dev/ADR_heatercontrol'
HCDBG: Port /dev/ttyUSB2 incorrect msg:   
HCDBG: Port /dev/ttyUSB2 incorrect msg: 
HCDBG: Port /dev/ttyUSB2 incorrect msg: I am HeaterController: 1E 95 F
HCDBG: Port /dev/ttyUSB2 incorrect msg: Type: A0
HCDBG: Port /dev/ttyUSB2 incorrect msg: FW: 12
HCDBG: Port /dev/ttyUSB2 incorrect msg: SN: 11
HCDBG: Port /dev/ttyUSB2 incorrect msg: Freq: 16000000
HCDBG: Port /dev/ttyUSB2 incorrect msg: Checking EEPROM...TYPE OK...SN OK!
HCDBG: Port /dev/ttyUSB2 incorrect msg: Timer 1 setup: TOP=31250 and f=4
HCDBG: Port /dev/ttyUSB2 CORRECT msg

In [ ]:
print(HC.deviceportname)

In [3]:
if (debug):
    print('HCDBG: Device type is {}'.format(HC.get_type()))
    print('HCDBG: Device serial is {}'.format(HC.get_serialnum()))
    print('HCDBG: Device status is {}'.format(HC.get_onoff()))
    print('HCDBG: Device duty cycle is {}'.format(HC.get_dutycycle()))

HCDBG: Device type is A0
HCDBG: Device serial is 11
HCDBG: Device status is 0
HCDBG: Device duty cycle is 0


In [ ]:
P = 1.0
I = 0.0
D = 0.0
pid = ADR_PID(P,I,D,0)
feedback = 0
feedback_list = []
time_list = []
setpoint_list = []
pid.SetPoint = 0.0
pid.setSlewLimit(0.0)
for i in range(1,100):
    if i>9:
        pid.SetPoint = 15
    pid.update(feedback)
    output = pid.output
    if pid.SetPoint > 0:
        feedback += (output - 1/i)
    time.sleep(0.02)
    feedback_list.append(feedback)
    setpoint_list.append(pid.SetPoint)
    time_list.append(i)

In [ ]:
plt.figure(1)
plt.plot(time_list, feedback_list)
plt.plot(time_list, setpoint_list),
plt.xlim(0, 100)
plt.xlabel('time (s)'),
plt.ylabel('PID (PV)')
plt.grid(True)
plt.show()

In [ ]:
    "plt.figure(2)\n",
    "plt.plot(time_list, [(feedback_list[i] - setpoint_list[i]) for  i in range(0,len(feedback_list))])\n",
    "plt.xlim((0, END))\n",
    "#plt.ylim((min(feedback_list)-0.5, max(feedback_list)+0.5))\n",
    "#plt.ylim((0,3))\n",
    "plt.xlabel('time (s)')\n",
    "plt.ylabel('PID (PV)')\n",
    "plt.title('TEST PID')\n",
    "\n",
    "#plt.ylim((1-0.5, 1+0.5))\n",
    "plt.ylim((0,3))\n",
    "\n",
    "plt.grid(True)\n",
    "plt.show()"

In [4]:
#Testing block
if(run_tests): 
    HC.flush_serial()
    HC.set_dutycycle(0)
    HC.set_onoff(0)
    commands = ['ON0000','OFF000','ON0000','SC0000','SC9999','SC0000','SC1000',\
              'SC0000','SCFULL','OFF000','ON0000','OFF000','SC0000','ON0000',\
              'SC0050','SC1000','SC3000','SC5000','SC7000','SC9950','SC5000']
    expected_s = [1,0,1,1,1,1,1,\
                  1,1,0,1,0,0,1,\
                  1,1,1,1,1,1,1,\
                  1,1,1,1,1,1,1]
    expected_c = [0,0,0,0,9999,0,1000,\
                  0,9999,9999,9999,9999,0,0,\
                  50,1000,3000,5000,7000,9950,5000,\
                  5000,5000,5000,5000,5000,5000,5000]
    print('HCDBG:    Command {:2} | S{:1} | C{:4}'.format('','','',''))
    for i,cmd in enumerate(commands):  
        if (cmd == 'ON0000'):
            HC.set_onoff(1)
        elif (cmd == 'OFF000'):
            HC.set_onoff(0)
        elif (cmd.startswith('SC')):
            if (cmd[2:] == 'FULL'):
                HC.set_dutycycle(100) 
            elif (cmd.endswith('0000')):
                HC.set_dutycycle(0) 
            else:
                HC.set_dutycycle(float(int(cmd[2:])+1)/100.0) 
        resp2 = HC.get_onoff()
        resp3 = HC.get_dutycycle()
        print('HCDBG:{:2}| {:10} | {:2} | {:4}'.format(i,cmd,resp2,resp3))
        if (resp2 != str(expected_s[i])):
            print('HCDBG: Wrong S {}, wanted {}'.format(resp2,expected_s[i]))
        if (resp3 != str(expected_c[i])):
            print('HCDBG: Wrong C {}, wanted {}'.format(resp3,expected_c[i]))
        time.sleep(1)
    HC.set_dutycycle(0)
    HC.set_onoff(0)

HCDBG:    Command    | S  | C    
HCDBG: 0| ON0000     | 1  | 0   
HCDBG: 1| OFF000     | 0  | 0   
HCDBG: 2| ON0000     | 1  | 0   
HCDBG: 3| SC0000     | 1  | 0   
HCDBG: 4| SC9999     | 1  | 9999
HCDBG: 5| SC0000     | 1  | 0   
HCDBG: 6| SC1000     | 1  | 1000
HCDBG: 7| SC0000     | 1  | 0   
HCDBG: 8| SCFULL     | 1  | 9999
HCDBG: 9| OFF000     | 0  | 9999
HCDBG:10| ON0000     | 1  | 9999
HCDBG:11| OFF000     | 0  | 9999
HCDBG:12| SC0000     | 0  | 0   
HCDBG:13| ON0000     | 1  | 0   
HCDBG:14| SC0050     | 1  | 50  
HCDBG:15| SC1000     | 1  | 1000
HCDBG:16| SC3000     | 1  | 3000
HCDBG:17| SC5000     | 1  | 5000
HCDBG:18| SC7000     | 1  | 7000
HCDBG:19| SC9950     | 1  | 9950
HCDBG:20| SC5000     | 1  | 5000
